In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.graphs import Neo4jGraph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)
graph.refresh_schema()
print(graph.schema)

Node properties are the following:

Relationship properties are the following:

The relationships are the following:



In [3]:
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

from loader.radiopaedia import RadioWebLoader

search_query = "atelectasis"
loader = RadioWebLoader(search_query)
raw_documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents(
    raw_documents
)  # should I only take the first? which is the most relevant?


In [4]:
# from langchain.document_loaders import DiffbotLoader

# raw_documents = DiffbotLoader(
#     urls=all_address,
#     api_token=DIFFBOT_API_TOKEN,
# ).load()
# text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
# documents = text_splitter.split_documents(
#     raw_documents
# )  # should I only take the first? which is the most relevant?

In [5]:
db = Neo4jVector.from_documents(
    documents, OpenAIEmbeddings(), url=NOE4J_URL, username="neo4j", password="test1234"
)

retriever = db.as_retriever()
# 4s

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.OpenAIEmbeddings instead.
  warn_deprecated(


In [6]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

In [7]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"), chain_type="stuff", retriever=retriever
)

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [8]:
lesion = "Atelectasis"
questions = [
    f"What is {lesion}?",
    f"What are the symptoms associated with {lesion}?",
    f"What can cause {lesion}?",
    f"What are the patient’s symptoms that are relevant for {lesion}?",
    f"What are the relevant clinical signs for the etiological diagnosis of {lesion}?",
    f"What are the relevant laboratory data for the etiological diagnosis of {lesion}?",
    f"What are the relevant clinical characteristics for the etiological diagnosis of {lesion}",
    f"What are the patient’s personal relevant history for the etiological diagnosis of {lesion}",
]

In [9]:
res_dict = {}
for q in questions:
    res = chain(
        {"question": q},
        return_only_outputs=True,
    )
    res_dict[q] = res["answer"].strip()

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [10]:
for q in questions:
    print(res_dict[q])

Atelectasis refers to collapse or incomplete expansion of pulmonary parenchyma. It can be caused by various factors, including resorptive (obstructive) atelectasis, passive (relaxation) atelectasis, compressive atelectasis, cicatrisation atelectasis, adhesive atelectasis, and gravity-dependent atelectasis. Passive atelectasis, also known as relaxation atelectasis, occurs when there is a loss of negative pressure in the pleural space, causing the lung to relax back to its normal volume. It can be caused by conditions such as simple pneumothorax, diaphragmatic abnormalities, and conditions of hypoventilation. The clinical presentation of atelectasis depends on the extent and speed of development, ranging from asymptomatic to features of hypoxia and respiratory failure. Radiographic features of atelectasis include small volume linear shadows on chest x-rays and a more accurate depiction on CT scans.
The symptoms associated with Atelectasis include breathlessness, cough, and chest pain. Ho

In [11]:
raise StopIteration()

StopIteration: 

In [ ]:
chain(
    {"question": "What is Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis refers to collapse or incomplete expansion of pulmonary parenchyma. It can be classified based on underlying mechanism (resorptive, passive, compressive, cicatrisation, adhesive, gravity dependent) or by morphology (linear, round, lung, lobar, segmental, subsegmental). The radiographic features of atelectasis vary depending on the underlying mechanism and type. It is better appreciated on CT scans. Segmental atelectasis specifically refers to the collapse of one or several segments of a lung lobe. \n',
 'sources': ''}

In [ ]:
chain(
    {"question": "What are the symptoms of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'The symptoms of Atelectasis can vary depending on the underlying cause. Common symptoms include breathlessness and cough, but it can also be asymptomatic. The presentation of symptoms also depends on the extent and speed of development of atelectasis. Physical examination may reveal diminished movement of the affected area, dullness on percussion, decreased or absent breath sounds, and deviation of the trachea. Treatment involves managing the underlying cause and may include deeper breathing exercises. Imaging, such as chest x-ray or CT, can confirm the presence of atelectasis and help determine the cause. \n',
 'sources': ''}

In [ ]:
chain(
    {"question": "What can cause Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis can be caused by various factors, including adjacent compression (such as a lung tumor or dilated aorta), passive atelectasis (when the lung relaxes away from the pleural surface), dependent atelectasis (due to patients not fully expanding their lungs while lying for long periods), and adhesive atelectasis (due to a decrease or absence of pulmonary surfactant). Other causes include simple pneumothorax, diaphragmatic abnormalities, conditions of hypoventilation, pleural effusion, and chest wall mass lesions. Cicatrisation atelectasis can occur as a result of scarring or fibrosis. \n',
 'sources': ''}

In [ ]:
chain(
    {"question": "What clinical features radiologists need for diagnosing Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Radiologists need to look for the following clinical features for diagnosing Atelectasis:\n- Small volume linear shadows on chest x-rays, usually peripherally or at the lung bases.\n- Triangular opacity in the posteromedial aspect of the left lung (retrocardiac sail sign) for left lower lobe collapse.\n- Linear bands radiating from a mass into adjacent lung tissue resembling the feet of a crow for round atelectasis.\n- Loss of negative intrapleural pressure leading to relaxation of the lung and possible hypoxia and respiratory failure for passive atelectasis.\n',
 'sources': ''}

In [ ]:
chain(
    {"question": "what are radiographic features of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Radiographic features of atelectasis include small volume linear shadows on chest x-rays, usually peripherally or at the lung bases. CT scans can provide a more accurate depiction of the involved lung and can help determine the cause of atelectasis. Other radiographic features may include displacement of interlobar fissures, crowding of pulmonary vessels and bronchi, and compensatory hyperexpansion of the surrounding or contralateral lung. Linear atelectasis, also known as Fleischner lines, may appear as thin, linear densities in the lung bases parallel to the diaphragm. Segmental atelectasis refers to the collapse of one or several segments of a lung lobe. The crow feet sign is a characteristic feature seen in round atelectasis on CT scans, appearing as linear bands radiating from a mass into adjacent lung tissue. \n',
 'sources': ''}